# After half a day with chatGPT, giving up. 

Interpolating the lat-lon location, and the Tb values pixelwise, all with two half-hourly grids to an hourly file with a single lat-lon extent, and then all on a canvas (paper, pixel grid) was too much for ol Chattie. It would either jump in space, or fail to interpolate in half the hour, round and round, failure after failure. 

In [1]:
# !mkdir /Users/bmapes/Box/Sky_Symphony_Box/IRMA2017/MERGEDIR_BOXES/frames2

In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import pandas as pd 
from pathlib import Path
import os
from subprocess import run
from matplotlib.colors import LinearSegmentedColormap, to_rgba
import matplotlib.colors as mcolors
from matplotlib.image import imread

In [3]:
# Read in huge Blue Marble image 
# --- Place this code BEFORE your 1000-frame loop (Load ONCE) ---

# Disable the decompression bomb check
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

# --- Configuration (MUST match your image and desired georeference) ---
STATIC_MAP_FILE = '/Users/bmapes/Box/Sky_Symphony_Box/BlueMarble.200408.3x21600x10800.jpg' 
FULL_WORLD_EXTENT = [-180, 180, -90, 90] 
try:
    # Load the single satellite image file ONLY ONCE
    BASE_IMAGE_DATA = imread(STATIC_MAP_FILE)
except FileNotFoundError:
    raise FileNotFoundError(f"Error: Static map file '{STATIC_MAP_FILE}' not found. Please download a high-res Blue Marble image and save it to this name.")

In [4]:
# ==== USER CONFIG ====  # frames3 has latest color scale, alpha never 0 
data_folder = Path("/Users/bmapes/Box/Sky_Symphony_Box/IRMA2017/MERGEDIR_BOXES/ncfiles/") 
frame_folder =Path("/Volumes/Samsung USB/interpframes_bluemarble/")
frame_folder.mkdir(exist_ok=True)

dpi = 150
# =====================

frame_folder.mkdir(exist_ok=True)

# 2. Preload lat/lon grids from all files
nc_files = sorted(data_folder.glob("*.nc"))
files_info = []
for file in nc_files:
    with xr.open_dataset(file) as ds:
        files_info.append({
            "file": file,
            "lat": ds['lat'].load(),
            "lon": ds['lon'].load()
        })

print('Got all files_info (lat-lon for interpolations)')

Got all files_info (lat-lon for interpolations)


In [5]:
# Custom color scale, including transparency to see Blue Marble underneath  
color_points = [
    (190, "#dc05ef", 1.0),  # magenta 
    (222, "#0589ef", 1.0),  # blue (color enhancement ends)
    (240, "#00ffff", 1.0),  # cyan
    (250, "#716f6f", 1.0),  # darker gray 
    (270, "#c5c6c6", 1.0),  # light-mid gray
    (280, "#ffffff", 0.9),  # white semitrans light gray
    (290, "#ffffff", 0.6),  # white point surface
    (300, "#ffffff", 0.1),  # SST point water
    (305, "#ffffff", 0.4),  # bush green cool surface --> WHITE
    (310, "#ff8000", 1.0),  # HOT surface orange, see https://html-color.codes/
    (340, "#000000", 1.0)  # HOTHOT surface black
]

# Brigher colors from BlueMarble (less white haze at high T), less garish hot orange
color_points = [
    (190, "#dc05ef", 1.0),  # magenta 
    (222, "#0589ef", 1.0),  # blue (color enhancement ends)
    (240, "#00ffff", 1.0),  # cyan
    (250, "#716f6f", 1.0),  # darker gray 
    (270, "#c5c6c6", 1.0),  # light-mid gray
    (280, "#ffffff", 0.7),  # white semitrans light gray
    (290, "#ffffff", 0.3),  # white point surface
    (300, "#ffffff", 0.1),  # SST point water
    (305, "#ffffff", 0.3),  # --> WHITE 
    (310, "#ff8000", 0.6),  # HOT surface orange, see https://html-color.codes/
    (340, "#000000", 0.8)  # HOTHOT surface black
]

vmin = 190; vmax=340 

cmap = LinearSegmentedColormap.from_list("custom", [
    ((v - vmin) / (vmax - vmin), to_rgba(c, a)) for v, c, a in color_points
])

In [1]:
!rm /Volumes/Samsung\ USB/interpframes_bluemarble/*png

In [7]:
# --- Safe restart logic for 2 times per file, 8 interpolated frames per time ---
existing = sorted([int(f.stem) for f in frame_folder.glob("*.png") if f.stem.isdigit()])

if existing:
    last_done_frame = max(existing)
    # Each timestep produces interp_factor frames (8)
    last_done_timestep = last_done_frame // interp_factor
    # Back up one timestep to be safe (catch any partially generated frames)
    restart_timestep = max(last_done_timestep - 1, 0)
    # Compute corresponding frame number to start writing at
    frame_count = restart_timestep * interp_factor
    print(f"Resuming from timestep {restart_timestep}, frame {frame_count}")
else:
    restart_timestep = 0
    frame_count = 0
    print(f"Starting from the beginning")
    done_frame_count = 0
# Use restart_timestep as the starting index in your main loop:
# for t_idx in range(restart_timestep, n_timesteps):


Starting from the beginning


In [ ]:
# --- Configuration ---
interp_factor = 8  # Number of frames per time step
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
STATIC_MAP_FILE = '/Users/bmapes/Box/Sky_Symphony_Box/BlueMarble.200408.3x21600x10800.jpg'
FULL_WORLD_EXTENT = [-180, 180, -90, 90] 
BASE_IMAGE_DATA = imread(STATIC_MAP_FILE)

# --- Prepare frame counters ---
existing = sorted([int(f.stem) for f in frame_folder.glob("*.png") if f.stem.isdigit()])
if existing:
    frame_count = max(existing) + 1
else:
    frame_count = 0

# --- Main loop over files ---
for i, file in enumerate(files_info):
    ds = xr.open_dataset(file['file'])
    Tb_all = ds['Tb']
    times = Tb_all.time.values
    lat_now, lon_now = file['lat'], file['lon']
    
    # Determine next lat/lon box for interpolation
    if i < len(files_info) - 1:
        lat_next = files_info[i + 1]['lat']
        lon_next = files_info[i + 1]['lon']
    else:
        lat_next, lon_next = lat_now, lon_now

    # Loop over time steps in the current file
    for t in range(Tb_all.sizes['time']):
        Tb = Tb_all.isel(time=t)
        timestamp_now = times[t]
        if t < Tb_all.sizes['time'] - 1:
            Tb_next = Tb_all.isel(time=t + 1)
            timestamp_next = times[t + 1]
        elif i < len(files_info) - 1:
            # Next file, first time step
            ds_next = xr.open_dataset(files_info[i + 1]['file'])
            Tb_next = ds_next['Tb'].isel(time=0)
            timestamp_next = ds_next['Tb'].time.values[0]
        else:
            Tb_next = Tb
            timestamp_next = timestamp_now

        # Interpolate interp_factor frames per time step
        for k in range(interp_factor):
            f = k / (interp_factor - 1)
            f_smooth = f * f * (3 - 2 * f)  # smootherstep

            # Interpolate Tb values
            Tb_interp = (1 - f_smooth) * Tb + f_smooth * Tb_next

            # Interpolate map extent
            coast_latmin = (1 - f_smooth) * np.min(lat_now) + f_smooth * np.min(lat_next)
            coast_lonmin = (1 - f_smooth) * np.min(lon_now) + f_smooth * np.min(lon_next)
            coast_latmax = (1 - f_smooth) * np.max(lat_now) + f_smooth * np.max(lat_next)
            coast_lonmax = (1 - f_smooth) * np.max(lon_now) + f_smooth * np.max(lon_next)
            extent = [float(coast_lonmin), float(coast_lonmax),
                      float(coast_latmin), float(coast_latmax)]

            # --- Plotting ---
            fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})
            ax.set_extent(extent, crs=ccrs.PlateCarree())
            ax.imshow(BASE_IMAGE_DATA, origin='upper', transform=ccrs.PlateCarree(), extent=FULL_WORLD_EXTENT)
            ax.coastlines()

            ts_str = str(np.datetime64(timestamp_now) + np.timedelta64(int(f * ((timestamp_next - timestamp_now)/1e9)), 's'))[:16]
            ts_str = ts_str.replace(":", "").replace("T", "_")
            ax.set_title(f"Interpolated Tb {ts_str} +{k}/{interp_factor}", fontsize=12)

            Tb_interp.interpolate_na(dim="lon", method="linear", max_gap=2).plot.imshow(
                ax=ax, transform=ccrs.PlateCarree(),
                cmap=cmap, vmin=vmin, vmax=vmax, add_colorbar=False
            )

            fig.subplots_adjust(top=0.92, bottom=0.05, left=0.05, right=0.98)
            out_file = frame_folder / f"{frame_count:08d}.png"
            plt.savefig(out_file, dpi=dpi)
            plt.close()
            frame_count += 1


In [ ]:
# Custom color scale, including transparency to see Blue Marble underneath  
# THIS ONE WAS A BIT GAUZY, MAKE MORE TRANSPARENCY AT LOW LEVELS / HIGH TB

color_points = [
    (190, "#dc05ef", 1.0),  # magenta 
    (222, "#0589ef", 1.0),  # blue (color enhancement ends)
    (240, "#00ffff", 1.0),  # cyan
    (250, "#716f6f", 1.0),  # darker gray 
    (270, "#c5c6c6", 1.0),  # light-mid gray
    (280, "#ffffff", 0.9),  # white semitrans light gray
    (290, "#ffffff", 0.6),  # white point surface
    (300, "#ffffff", 0.1),  # SST point water
    (305, "#ffffff", 0.4),  # bush green cool surface --> WHITE
    (310, "#ff8000", 1.0),  # HOT surface orange, see https://html-color.codes/
    (340, "#000000", 1.0)  # HOTHOT surface black
]

vmin = 190; vmax=340 

cmap = LinearSegmentedColormap.from_list("custom", [
    ((v - vmin) / (vmax - vmin), to_rgba(c, a)) for v, c, a in color_points
])

In [ ]:
# Test colors for one image 

data_folder = Path("/Users/bmapes/Box/Sky_Symphony_Box/IRMA2017/MERGEDIR_BOXES/ncfiles/")  
nc_files = sorted(data_folder.glob("*.nc"))

In [ ]:
ds = xr.open_dataset(nc_files[200])  # 21Z = 18+24-15 cold 
ds = xr.open_dataset(nc_files[200-12 +24*0])      # Sahel coast crossing 

Tb = ds.Tb[0]

# Define extent from coast file
extent = [ds.lon.min().values, ds.lon.max().values,ds.lat.min().values, ds.lat.max().values]

# Plot data on top of map image, there is some transparency 
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_extent(extent, crs=ccrs.PlateCarree())

# Background image: stock image, or slow web call, tiles keeps crashing on me, sigh
ax.imshow(BASE_IMAGE_DATA,
  origin='upper',
  transform=ccrs.PlateCarree(),
  extent=FULL_WORLD_EXTENT)

ax.coastlines()

Tb.interpolate_na(dim="lon", method="linear", max_gap=2).plot.imshow(
    ax=ax, transform=ccrs.PlateCarree(), cmap=cmap, vmin=vmin,vmax=vmax,\
    add_colorbar=True, #cbar_kwargs={'label': 'Brightness Temperature [K]'}
)
plt.tight_layout()


In [ ]:
# Custom color scale, including transparency to see Blue Marble underneath  
color_points = [
    (190, "#dc05ef", 1.0),  # magenta 
    (222, "#0589ef", 1.0),  # blue (color enhancement ends)
    (240, "#00ffff", 1.0),  # cyan
    (250, "#716f6f", 1.0),  # darker gray 
    (270, "#c5c6c6", 1.0),  # light-mid gray
    (280, "#ffffff", 0.7),  # white semitrans light gray
    (290, "#ffffff", 0.3),  # white point surface
    (300, "#ffffff", 0.1),  # SST point water
    (305, "#ffffff", 0.3),  # --> WHITE 
    (310, "#ff8000", 0.6),  # HOT surface orange, see https://html-color.codes/
    (340, "#000000", 0.8)  # HOTHOT surface black
]

vmin = 190; vmax=340 
cmap = LinearSegmentedColormap.from_list("custom", [
    ((v - vmin) / (vmax - vmin), to_rgba(c, a)) for v, c, a in color_points
])

ds = xr.open_dataset(nc_files[200-12 +24*17 +8])  # End of movie afternoon 
ds = xr.open_dataset(nc_files[200-12 +24*0])      # Sahel coast crossing 

Tb = ds.Tb[0]
# Define extent from datafile
extent = [ds.lon.min().values, ds.lon.max().values,ds.lat.min().values, ds.lat.max().values]


# Plot data on top of map image, there is some transparency 
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_extent(extent, crs=ccrs.PlateCarree())

# Background image: stock image, or slow web call, tiles keeps crashing on me, sigh
ax.imshow(BASE_IMAGE_DATA,
  origin='upper',
  transform=ccrs.PlateCarree(),
  extent=FULL_WORLD_EXTENT)

ax.coastlines()

Tb.interpolate_na(dim="lon", method="linear", max_gap=2).plot.imshow(
    ax=ax, transform=ccrs.PlateCarree(), cmap=cmap, vmin=vmin,vmax=vmax,\
    add_colorbar=True, #cbar_kwargs={'label': 'Brightness Temperature [K]'} 
);
plt.tight_layout()